In [ ]:
!pwd

'/content'

In [3]:
# import libraries
import torch
import torch.nn as nn
from PIL import Image
import numpy as np

def flatten_image(image):
  """Flattens a PIL image to a 1D NumPy array.

  Args:
    image: A PIL image object.

  Returns:
    A 1D NumPy array containing the pixel values of the image.
  """

  image = image.convert('L') # convert to grayscale
  image_array = np.array(image) # convert to NumPy array
  flattened_image = image_array.flatten() # flatten the image
  return flattened_image


# Define the MLP architecture
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()

        # Input layer
        self.input = nn.Linear(1024, 256) # 32*32 = 1024

        # Hidden layer
        self.hidden = nn.Linear(256, 256) # 256 nodes

        # Output layer
        self.output = nn.Linear(256, 3) # 3 nodes for multi-class classification

        # Activation function
        self.activation = nn.ReLU()

    def forward(self, x):
        x = self.input(x)
        x = self.activation(x)
        x = self.hidden(x)
        x = self.activation(x)
        x = self.output(x)

        return x

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # use GPU if available

# Load the training images
images = []
for i in range(1, 4):
    image = Image.open(f'Q5_{i}.png') # /content/Q5_{i}.png
    # print(image)
    image = image.resize((32, 32)) # resize to 32x32

    # image = image.flatten()
    # Flatten the image
    flattened_image = flatten_image(image) # convert to 1D array (1024,) convert to grayscale and flatten

    images.append(flattened_image)

# Convert the images to PyTorch tensors
images = torch.tensor(np.array(images), dtype=torch.float32).to(device)

# Load the training labels
labels = torch.tensor(np.array([0, 1, 2]), dtype=torch.long).to(device) # {'s':0, 'g':1, 'b':2}

# Create an instance of the MLP network
model = MLP().to(device)

# Define the loss function
criterion = nn.CrossEntropyLoss()

# Define the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.001) # Adam: Adaptive moment estimation

model.train() # set the model to training mode
# Train the model
for epoch in range(100):
    # Forward pass
    outputs = model(images)

    # Calculate the loss
    loss = criterion(outputs, labels) # CrossEntropyLoss

    # Backward pass
    optimizer.zero_grad()
    loss.backward() # d Loss / d parameters
    optimizer.step() # run gradiant decent

    # print training information
    if epoch % 5 == 0:
        print('Epoch:', epoch, 'Loss:', loss.item())

    # print accuracy
    if epoch % 5 == 0:
        _, predicted_labels = torch.max(outputs, dim=1)
        accuracy = (predicted_labels == labels).sum() / len(labels)
        print('Trainig Accuracy:', accuracy.item())
        print("--------------------------------")


# Evaluate the model on the test images (test are same as training because we only 3 examples)
model.eval() # set the model to evaluation mode
with torch.no_grad():
  outputs = model(images)
  _, predicted_labels = torch.max(outputs, dim=1)

  accuracy = (predicted_labels == labels).sum() / len(labels)

  print('Final Test Accuracy:', accuracy.item())


Epoch: 0 Loss: 10.962074279785156
Trainig Accuracy: 0.3333333432674408
--------------------------------
Epoch: 5 Loss: 5.654520034790039
Trainig Accuracy: 0.6666666865348816
--------------------------------
Epoch: 10 Loss: 2.910238027572632
Trainig Accuracy: 0.6666666865348816
--------------------------------
Epoch: 15 Loss: 1.7841179214883596e-05
Trainig Accuracy: 1.0
--------------------------------
Epoch: 20 Loss: 7.947276685627003e-07
Trainig Accuracy: 1.0
--------------------------------
Epoch: 25 Loss: 3.973642748178463e-08
Trainig Accuracy: 1.0
--------------------------------
Epoch: 30 Loss: 0.0
Trainig Accuracy: 1.0
--------------------------------
Epoch: 35 Loss: 0.0
Trainig Accuracy: 1.0
--------------------------------
Epoch: 40 Loss: 0.0
Trainig Accuracy: 1.0
--------------------------------
Epoch: 45 Loss: 0.0
Trainig Accuracy: 1.0
--------------------------------
Epoch: 50 Loss: 0.0
Trainig Accuracy: 1.0
--------------------------------
Epoch: 55 Loss: 0.0
Trainig Accura